In [3]:
inv_index_file_name = "ru_inverted_index.txt"
files_total_count = 195

In [4]:
all_files_set = set(range(1, files_total_count))
all_files_set_var_name = "all_files_set"

In [5]:
class InvertedIndexEntry:
    def __init__(self, lemma, count, files):
        self.lemma = lemma
        self.count = count
        self.files = files
    
    def __repr__(self):
        return f"InvertedIndexEntry(lemma = '{self.lemma}', count = {self.count}, files = {sorted(self.files)})"

In [6]:
inv_index = dict()
inv_index_var_name = "inv_index"

In [7]:
import re

with open(inv_index_file_name, 'r', encoding="utf-8") as file:
    lines = file.readlines()
    for line in lines:
        pattern = r'(\w+)\[(\d+)\]\s+([\w\s]+)'
        match = re.match(pattern, line)
        
        lemma = match.group(1)
        count = int(match.group(2))
        files = [int(num) for num in match.group(3).split(' ')]

        inv_index_entry = InvertedIndexEntry(lemma, count, set(files))
        
        inv_index[lemma] = inv_index_entry

In [8]:
# порядок операций по важности 
# () ! & |
CLOSING_BRACE = ')'
OPENING_BRACE = '('

OR_QEURY = '|'
NOT_QEURY = '!'
AND_QEURY = '&'

OR_REPLACE = ").union("
NOT_REPLACE = ").difference("
AND_REPLACE = ").intersection("

In [15]:


def next_is_word(current_i, query_words):
    next_i = current_i + 1
    
    if next_i == len(query_words):
        return False
        
    next_word = query_words[next_i]

    return not next_word in (OR_QEURY, NOT_QEURY, AND_QEURY, CLOSING_BRACE, OPENING_BRACE)

def boolean_search(initial_query, inverted_index, debug=False):
    initial_query = initial_query.replace(OPENING_BRACE, f" {OPENING_BRACE} ").replace(CLOSING_BRACE, f" {CLOSING_BRACE} ")
    initial_query = initial_query.replace(OR_QEURY, f" {OR_QEURY} ").replace(NOT_QEURY, f" {NOT_QEURY} ").replace(AND_QEURY, f" {AND_QEURY} ")
    query_words = initial_query.strip().lower().split(' ')
    query_words = list(filter(lambda x: x != "", query_words))

    i = 0
    query = OPENING_BRACE

    if debug:
        print(query_words)
    
    for word in query_words:
        if word == '':
            # skip
            query = query
        elif word == OPENING_BRACE:
            query += OPENING_BRACE
        elif word == CLOSING_BRACE:
            query += CLOSING_BRACE
        elif word == NOT_QEURY:
            if next_is_word(i, query_words):
                query += f'{all_files_set_var_name}.difference('
            else:
                query += NOT_REPLACE
        elif word == AND_QEURY:
            query += AND_REPLACE
        elif word == OR_QEURY:
            query += OR_REPLACE
        else:
            # замена слова в запросе с сетом файлов из инвертированного индекса
            token = word
            if debug:
                print()
                print(token)
            if (token in inv_index.keys()):
                if debug:
                    print(sorted(eval(f'inv_index["{token}"].files')))
                query += f'{inv_index_var_name}["{token}"].files'
            else:
                if debug:
                    print(sorted(eval(f'set()')))
                query += f'set()'
        i += 1
                
    query += CLOSING_BRACE
    
    if debug:
        print()
        print(query)
        print()

    # йесссс юзаем eval()
    result = eval(query)
    return sorted(result)

In [16]:
test_query = "анализ & (делать | чуткий)"
result = boolean_search(test_query, inv_index, debug=True)
print(f'Search result = {result}')

['анализ', '&', '(', 'делать', '|', 'чуткий', ')']

анализ
[3, 4, 14, 15, 19, 25, 27, 31, 38, 56, 66, 82, 91, 102, 104, 109, 112, 113, 121, 122, 130, 150, 154, 171, 194]

делать
[3, 4, 7, 10, 14, 18, 20, 22, 25, 26, 28, 31, 32, 33, 35, 36, 37, 39, 40, 47, 49, 50, 53, 56, 61, 67, 70, 74, 77, 85, 86, 88, 97, 98, 103, 104, 109, 111, 113, 116, 119, 121, 123, 125, 130, 132, 137, 147, 148, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 167, 169, 170, 171, 173, 175, 176, 178, 183, 184, 185, 188, 191, 192, 193]

чуткий
[184]

(inv_index["анализ"].files).intersection((inv_index["делать"].files).union(inv_index["чуткий"].files))

Search result = [3, 4, 14, 25, 31, 56, 104, 109, 113, 121, 130, 154, 171]


In [14]:
test_query_2 = "!анализ & (делать | чуткий)"
result_2 = boolean_search(test_query_2, inv_index, debug=True)
print(f'Search result = {result_2}')

['!', 'анализ', '&', '(', 'делать', '|', 'чуткий', ')']

анализ
[3, 4, 14, 15, 19, 25, 27, 31, 38, 56, 66, 82, 91, 102, 104, 109, 112, 113, 121, 122, 130, 150, 154, 171, 194]

делать
[3, 4, 7, 10, 14, 18, 20, 22, 25, 26, 28, 31, 32, 33, 35, 36, 37, 39, 40, 47, 49, 50, 53, 56, 61, 67, 70, 74, 77, 85, 86, 88, 97, 98, 103, 104, 109, 111, 113, 116, 119, 121, 123, 125, 130, 132, 137, 147, 148, 154, 155, 158, 159, 160, 161, 162, 163, 164, 165, 167, 169, 170, 171, 173, 175, 176, 178, 183, 184, 185, 188, 191, 192, 193]

чуткий
[184]

(all_files_set.difference(inv_index["анализ"].files).intersection((inv_index["делать"].files).union(inv_index["чуткий"].files)))

Search result = [7, 10, 18, 20, 22, 26, 28, 32, 33, 35, 36, 37, 39, 40, 47, 49, 50, 53, 61, 67, 70, 74, 77, 85, 86, 88, 97, 98, 103, 111, 116, 119, 123, 125, 132, 137, 147, 148, 155, 158, 159, 160, 161, 162, 163, 164, 165, 167, 169, 170, 173, 175, 176, 178, 183, 184, 185, 188, 191, 192, 193]


In [17]:
print("Enter query. Supported special characters: '&', '|', '!', '(', ')'")

query = str(input())
result = boolean_search(query, inv_index)
print(f'Search result = {result}')

Enter query. Supported special characters: '&', '|', '!', '(', ')'


 анализ & (делать | чуткий)


Search result = [3, 4, 14, 25, 31, 56, 104, 109, 113, 121, 130, 154, 171]
